6 Coding Exercise:


Identifying Defective Products in an E-Commerce Platform.

Should we Recall This Product?

Background:

You are a data analyst at Quick-shop, an e-commerce company. The customer support team has flagged
”Product X” for unusually high return rates. Before taking costly actions (e.g., recalls, supplier penalties),
you need to determine whether the returns are due to defects (quality issues) or other factors (e.g., customer
preferences).

Your team has provided a dataset with 10,000 products, including:
Dataset: "product\_review\_dataset.csv"

• Defective (Binary: 1 = defective, 0 = not defective)

• HighReturn (Binary: 1 = high return rate, 0 = low return rate)

• ReviewRating (Numeric: 1–5 star rating)

• HasComplaint (Boolean: True/False)

• VerifiedPurchase (Boolean: True/False)

Sample row:

Defective=0, HighReturn=1, ReviewRating=3.8, HasComplaint=False, VerifiedPurchase=True

Tasks
1. Exploratory Analysis
1. Compute the prior probability that a product is defective (P(Defective))
2. Compare the average review rating for defective vs. non-defective products
3. Calculate the return rate separately for:

  • Defective products
  
  • Non-defective products


In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import pandas as pd

file_path = '/content/drive/MyDrive/data analysis/product_review_dataset.csv'
df = pd.read_csv(file_path)


In [3]:
print(df.info)


<bound method DataFrame.info of       Defective  HighReturn  ReviewRating  HasComplaint  VerifiedPurchase
0             0           0           3.8         False              True
1             1           1           2.9         False              True
2             0           0           3.6         False              True
3             0           0           4.6         False              True
4             0           0           3.9         False              True
...         ...         ...           ...           ...               ...
9995          0           0           3.6         False             False
9996          0           0           4.1         False              True
9997          1           1           3.3         False              True
9998          0           0           4.1         False             False
9999          0           0           4.3         False              True

[10000 rows x 5 columns]>


In [4]:
print(df.describe())


          Defective    HighReturn  ReviewRating
count  10000.000000  10000.000000  10000.000000
mean       0.096100      0.159600      4.015620
std        0.294743      0.366253      0.712447
min        0.000000      0.000000      1.000000
25%        0.000000      0.000000      3.700000
50%        0.000000      0.000000      4.100000
75%        0.000000      0.000000      4.500000
max        1.000000      1.000000      5.000000


In [6]:
P_Defective = df['Defective'].mean()

avg_rating_defective = df[df['Defective'] == 1]['ReviewRating'].mean()
avg_rating_non_defective = df[df['Defective'] == 0]['ReviewRating'].mean()

return_rate_defective = df[df['Defective'] == 1]['HighReturn'].mean()
return_rate_non_defective = df[df['Defective'] == 0]['HighReturn'].mean()

P_Defective, avg_rating_defective, avg_rating_non_defective, return_rate_defective, return_rate_non_defective


(np.float64(0.0961),
 np.float64(2.4836628511966703),
 np.float64(4.178493196150017),
 np.float64(0.696149843912591),
 np.float64(0.10255559243279123))

In [7]:
# Task 2.1: Bayesian Inference
# P(HighReturn | Defective)
P_HighReturn_given_Defective = return_rate_defective

# P(HighReturn | Not Defective)
P_HighReturn_given_NotDefective = return_rate_non_defective

# P(HighReturn)
P_HighReturn = df['HighReturn'].mean()

# P(Defective | HighReturn) = P(HighReturn | Defective) * P(Defective) / P(HighReturn)
P_Defective_given_HighReturn = (P_HighReturn_given_Defective * P_Defective) / P_HighReturn

P_HighReturn, P_HighReturn_given_Defective, P_HighReturn_given_NotDefective, P_Defective_given_HighReturn


(np.float64(0.1596),
 np.float64(0.696149843912591),
 np.float64(0.10255559243279123),
 np.float64(0.4191729323308271))

**If P(Defective | HighReturn)= 42%**


This means that among products with high return rates, only about 42% are likely defective.

**Should QuickShop prioritize recalls based solely on this probability? Justify your answer.**

Quick-Shop should not recall products solely based on high return rates, since over half of high-return products are likely not defective.

In [8]:
# Task 3.1: Create a risk score combining HighReturn, LowRating, Complaints

# Define LowRating as ReviewRating <= 2
df['LowRating'] = df['ReviewRating'] <= 2

# Risk Score: sum of HighReturn, LowRating, HasComplaint (True = 1)
df['RiskScore'] = df['HighReturn'] + df['LowRating'].astype(int) + df['HasComplaint'].astype(int)

# Top 10 highest-risk products
top_10_risk = df.sort_values(by='RiskScore', ascending=False).head(10)

# Display relevant columns
top_10_risk[['Defective', 'HighReturn', 'ReviewRating', 'HasComplaint', 'RiskScore']]


,Defective,HighReturn,ReviewRating,HasComplaint,RiskScore
9983,1,1,1.5,True,3
7942,1,1,1.1,True,3
7924,1,1,1.6,True,3
8392,1,1,1.8,True,3
7817,1,1,2.0,True,3
7796,1,1,1.9,True,3
2954,1,1,1.9,True,3
2927,1,1,1.8,True,3
2529,1,1,1.9,True,3
2835,1,1,1.7,True,3


For a product with:

 • HighReturn=1

 • ReviewRating=1.5

 • HasComplaint=True

 • VerifiedPurchase=False

 Would you recommend a recall? Justify using your calculations.

Risk factor:

High return

Low rating

Has compliant

Risk Score is 3 which is very risk

What we know from the data is 42% of high return products are actually defective. Low review rating and complaints make defect suspicion even stronger.But lack of VerifiedPurchase means the review or complaint may be unreliable.

**Recommendations:**

A full recall for just this product is not justified solely on this data point, but further investigation is strongly recommended:

. Pull more customer service tickets.

. Inspect a sample batch.

. Check supplier quality reports.

If similar patterns are found across multiple verified purchases, then a targeted recall or supplier action is justified.



**Propose what additional data would improve this analysis**

For better evidence based decisions quick shop should collect:


*   Detailed return reasons (eg: wrong size, late delivery, wrong product)
*   Defect inspection reports from returned units


*   Batch or lot numbers to defect faulty production batches
*   Customer demographics to see if returns are tied to certain groups or locations.
*   Supplier/manufacturer defect history.
*    Verified purchase confirmations to weed out fake complaints.




